# Sous-différentiel et le cas du Lasso ☕️☕️☕️

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé&nbsp;:
    * À la notion d'optimisation via l'algorithme *proximal*,
    * À la notion de sous-gradient.
* Être capable d'implémenter à la main l'algorithme *proximal*.



 ----

## Introduction 

Soit $X\in\mathbb{R}^{n\times d}$ et $\boldsymbol{y}=\mathbb{R}^n$. De manière assez directe, le problème des moindres carrés se formule de la manière suivante :

$$\beta^\star=\text{argmin}_{\beta\in\mathbb{R}^d}\lVert X\beta-\boldsymbol{y}\rVert_2^2.$$

Supposons que $X^TX$ soit inversible. Alors, en annulant le gradient, on obtient :

$$\beta^\star=(X^TX)^{-1}X^T\boldsymbol{y}.$$

Si nous avions souhaité minimiser notre fonction objectif via une descente de gradient (par exemple dans le cas où $X^TX$ n'est pas inversible ou si le coût de son inversion est trop important), alors le gradient à utiliser est :

$$\nabla (\lVert X\beta-\boldsymbol{y}\rVert_2)=2X^TX-2X^t\boldsymbol{y}.$$

Cependant, comme nous l'avons vu dans une séquence précédente, il est parfois préférable de "régulariser" ce problème d'optimisation de manière à obtenir un vecteur $\beta^\star$ généralisant mieux à de nouveaux exemples d'apprentissage en machine learning ou offrant plus de stabilité. Le cas de la régularisation $\lVert\cdot\rVert_1$ dit $\ell_1$, qu'on appelle Lasso, est intéressant car il permet de faire de la sélection de variables. En effet, le vecteur $\beta^\star$ obtenu après minimisation est sparse.

Rappelons que la norme $\ell_1$ se définie de la manière suivante :

$$\lVert x \rVert_1=\sum_i |x_i|.$$

Le problème régularisé peut donc se reformuler de la manière suivante :

$$\beta^\star=\text{argmin}_{\beta\in\mathbb{R}^d}\lVert X\beta-\boldsymbol{y}\rVert_2^2+\lambda \lVert \beta\rVert_1,$$

où $\lambda\geq0$ contrôle la quantité de régularisation.

Le problème ici est que $\lVert \beta\rVert_1$ n'est pas différentiable partout - la valeur absolu n'est pas dérivable en $0$. On aurait pu se dire que ce n'est pas un problème et que notre descente de gradient aurait à coup sûr "sauté" les points non-différentiables, mais ces derniers s'avèrent justement être solution de notre problème : c'est le côté sparse. Nous allons, dans cette séquence introduire quelques éléments permettant d'aborder ce problème : le sous-différentiel.


À la fin de cette séquence, nous dériverons un algorithme permettant d'obtenir la solution du problème d'optimisation Lasso !

## I. Quelques bases

Nous allons ici introduire quelques éléments mathématiques permettant de dériver les algorithmes d'optimisation voulus. Tout d'abord, soit $f:\mathbb{R}\mapsto\mathbb{R}$ une fonction dérivable, alors sa tangente en $a$ a pour équation :

$$t(x)=f(a)+f^\prime(a)(x-a).$$

Si $f$ est convexe, alors $f(y)\geq t(x),\ \forall x, y\in\mathbb{R}$. Rappelons que la convexité d'une fonction $f$ nous indique que $\forall x,y\in\mathbb{R},\ \lambda\in [0, 1]$, on a :

$$f(\lambda x+(1-\lambda)y)\leq \lambda f(x)+(1-\lambda)f(y).$$

### La sous-dérivée
L'idée de la sous-dérivée est de pouvoir généraliser l'idée que la dérivée contrôle la tangente dans le cas d'une fonction convexe. Soit $f$ une fonction convexe, $s$ est une sous dérivée de $f$ en $a$ si :

$$f(x) \geq f(a)+s(x-a),\ \forall x\in\mathbb{R}.$$

La sous-dérivée n'est pas forcément unique en un point $a$ et on appelle sous-différentiel l'ensemble des sous-dérivés qu'on note $\partial f(a)$. Cependant, si $f$ est différentiable en $a$, alors $\partial f(a)=\{f^\prime(a)\}$. On observe assez rapidement que si $f$ est concave au voisinage de $a$, alors elle n'admet pas de sous-dérivée : $\partial f(x)=\emptyset$.

La figure suivante illustre quelques sous-dérivées pour les fonctions $f(x)=|x|$ et $g(x)=x^2$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(-2, 2, 101)


y = np.abs(x)
sub_gradient = np.stack([0.9*x, 0.3*x, -0.3*x, -0.7*x])

x = np.tile(x, (4, 1))

plt.figure(figsize=(16.0, 8.0))
plt.subplot(1, 2, 1)
plt.plot(x[0], y, label='$f(x)=|x|$')
lines = plt.plot(x.T, sub_gradient.T, 
                 label=r'$t_s(x)=\langle s, x\rangle$, $s$ sous-dérivée', color='gray', alpha=0.5)
plt.setp(lines[1:], label="_")

plt.ylim(-2, 2)

plt.gca().spines['left'].set_position('center')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['bottom'].set_position('center')
plt.gca().spines['top'].set_color('none')
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('left')

plt.legend()
plt.subplot(1, 2, 2)
plt.ylim(-2, 2)
plt.plot(x[0], x[0]**2, label='$g(x)=x^2$')
y = 0.5**2 + 2*0.5*(x[0]-0.5)
plt.plot(x[0], y, color='gray', alpha=0.5, label='Tangente')
plt.gca().spines['left'].set_position('center')
plt.gca().spines['right'].set_color('none')
plt.gca().spines['bottom'].set_position('center')
plt.gca().spines['top'].set_color('none')
plt.gca().xaxis.set_ticks_position('bottom')
plt.gca().yaxis.set_ticks_position('left')
plt.legend()
plt.show()

On observe bien l'infinité des sous-dérivées de la valeur absolue en $0$ et l'unicité en tout point pour la fonction $x^2$.

Un certain nombre de règles de calcul usuelles se généralisent assez bien à ce nouveau concept. Soit $f, g$ deux fonctions et un scalaire $\alpha$, nous avons:

$$\partial (\alpha(f+g))=\alpha\partial f+\alpha \partial g.$$

Soit $f$ une fonction dérivable et soit le problème d'optimisation suivant :

$$x^\star=\text{argmin}_{x\in\mathbb{R}}f(x).$$

La condition d'optimalité du premier ordre, dit de Fermat, nous donne l'équivalence suivante :

$$x^\star\text{ est un minimum local}\Leftrightarrow f^\prime(x^\star)=0.$$

Si $f$ est convexe, alors $x^\star$ est un minimum global. Qu'en est-il dans le cas où $f$ n'est pas dérivable en $x^\star$ mais sous-différentiable ? Supposons $f$ convexe. On a :

$$x^\star\text{ est un minimum global}\Leftrightarrow 0\in\partial f(x^\star).$$

Lorsqu'on considère des fonctions de $\mathbb{R}^n$ dans $\mathbb{R}$, alors les principes précédents se généralisent. On parle alors de sous-gradient.

## II. L'algorithme d'optimisation proximal et son application au Lasso

Reprenons le problème d'optimisation régularisé suivant :

$$\beta^\star=\text{argmin}_{\beta\in\mathbb{R}^d}\lVert X\beta-\boldsymbol{y}\rVert_2+\lambda \lVert \beta\rVert_1 = f(\beta)+g(\beta),$$

où $f$ et $g$ sont des fonctions convexes, $f$ une fonction différentiable.

Une stratégie d'optimisation de ce problème est ce qu'on appelle l'algorithme *proximal*. Ce dernier découpe chaque pas d'optimisation en deux étapes. La première consiste à optimiser $f$ en suivant son gradient (comme dans la descente de gradient). La seconde étape consiste à "optimiser $g$ dans le voisinage du point précédemment obtenu" :

$$\text{pas d'optimisation : }\begin{cases}u^{(t+1)}&=\beta^{(t)}-\gamma\nabla f(\beta^{(t)})\\
\beta^{(t+1)}&=\textbf{prox}_{\gamma g}(u^{(t+1)})\end{cases}$$

Cette algorithme s'appuie sur un opérateur qu'on appelle "l'opérateur proximal" :

$$\textbf{prox}_{\gamma g}(u)=\text{argmin}_y\big\{g(y)+\frac{1}{2\gamma}\lVert y-u\rVert_2^2\big\}.$$

On cherche à minimiser $g$ depuis un point $u$ en gardant la distance $\ell_2$ au carré faible.

**La pénalité $\ell_1$ du Lasso et l'opérateur proximal :** Soit $g(x)=\lambda \lVert x\rVert_1$. On cherche donc à trouver un point d'annulation de la sous-différentielle de la fonction objectif de notre opérateur proximal. On a donc :

$$\textbf{prox}_{\gamma g}(u)=\textbf{prox}_{\gamma \lambda \lVert \cdot\rVert_1}(u),$$

et en calculant la différentielle, nous obtenons :

$$\partial(\lVert y\rVert_1+\frac{1}{2\lambda\gamma}\lVert y-u\rVert_2^2)=\partial \lVert y\rVert_1+\frac{1}{2\lambda\gamma}\partial \lVert y-u \rVert_2^2=\partial \lVert y\rVert_1+\Big\{\frac{y-u}{\lambda\gamma}\Big\}.$$

Le problème est séparable et peut être considéré coordonnée par coordonnée :

$$(\partial \lVert y\rVert_1)_i=\begin{cases}1&\text{ si }y_i>0\\ [-1, 1]&\text{ si }y_i=0\\ -1&\text{ sinon.}\end{cases}$$

En combinant les deux sous-différentielles, et en traitant les deux problèmes coordonnée par coordonnée, nous obtenons :

$$\partial(\lVert y\rVert_1+\frac{1}{2\lambda\gamma}\lVert y-u\rVert_2^2)=\begin{cases}1+\frac{y_i-u_i}{\lambda\gamma}&\text{ si }y_i>0\\ \Big[-1+\frac{y_i-u_i}{\lambda\gamma}, 1+\frac{y_i-u_i}{\lambda\gamma}\Big]&\text{ si }y_i=0\\ -1+\frac{y_i-u_i}{\lambda\gamma}&\text{ si }y<0.\end{cases}$$

Notre objectif est de trouver $y_i$ tel que $0\in\partial(\lVert y\rVert_1+\frac{1}{2\lambda\gamma}\lVert y-u\rVert_2^2)$. Trois scénarios sont possibles. Tout d'abord :

$$0=1+\frac{y_i-u_i}{\lambda\gamma}\text{ et }y_i>0\Leftrightarrow y_i=u_i-\lambda\gamma\text{ et }u_i>\lambda\gamma.$$

Les deux autres scénarios se construisent exactement de la même manière. On obtient au final :

$$\textbf{prox}_{\gamma \lambda \lVert \cdot\rVert_1}(u)_i=\begin{cases}u_i-\lambda\gamma&\text{ si }u_i>\lambda\gamma\\ u_i+\lambda\gamma&\text{ si } u_i<-\lambda\gamma\\ 0&\text{ si }u_i\in[-\lambda\gamma,\lambda\gamma].\end{cases}$$

**La fonction objectif non régularisée du Lasso :** Soit $f(\beta)=\lVert X\beta-y\rVert_2^2$. On a comme nous l'avons déjà vu plusieurs fois :

$$\nabla f(\beta)=(X^TX)\beta-X^Ty.$$

**L'algorithme proximal appliqué au Lasso :** Soit le problème Lasso suivant : 

$$\beta^\star=\text{argmin}_{\beta\in\mathbb{R}^d}\lVert X\beta-\boldsymbol{y}\rVert_2+\lambda \lVert \beta\rVert_1 = f(\beta)+g(\beta).$$

L'algorithme proximal est ainsi un algorithme d'optimisation qui réalise une suite de pas d'optimisation. Un pas d'optimisation est construit de la manière suivante : 

$$\text{pas d'optimisation : }\begin{cases}u^{(t+1)}&=\beta^{(t)}-\gamma\nabla f(\beta^{(t)})\\
\beta^{(t+1)}&=\textbf{prox}_{\gamma \lambda \lVert \cdot\rVert_1}(u^{(t+1)})\end{cases}$$

## III. mise en pratique

### Construction du jeu de données

In [ ]:
real_beta = np.array([[8.], [-1.]])

def construct_dataset(n):
    global real_beta
    X = np.random.normal(0, 1, size=(n, 2))
    X[:, 0] += 2
    y = np.dot(X, real_beta) + np.random.normal(0, 1, size=(n, 1))
    return X, y
X, y = construct_dataset(5)
lambda_ = 2

### Les fonctions à optimiser

**<span style='color:blue'> Exercice</span>** **Donnez le code permettant évaluer la formule :**

$$\frac{1}{2}\lVert X\beta-y \rVert_2^2.$$

**On appellera cette fonction $\texttt{main}\_\texttt{objective}$.**



 ----

In [ ]:
def main_objective(X, y, beta):
    ####### Complete this part ######## or die ####################
    ...
    return ...
    ###############################################################


**<span style='color:blue'> Exercice</span>** 
**Donnez le code permettant évaluer la formule :**

$$\lambda\lVert \beta\rVert_1.$$

**On appellera cette fonction $\texttt{penalty}$.**



 ----

In [ ]:
def penality(beta, lambda_):
    ####### Complete this part ######## or die ####################
    ...
    return ...
    ###############################################################


**<span style='color:blue'> Exercice</span>** 
**En utilisant les deux fonctions précédentes, donnez le code permettant d'évaluer la formule :**

$$\frac{1}{2}\lVert X\beta-y \rVert_2^2+\lambda\lVert \beta\rVert_1.$$

**On appellera cette fonction $\texttt{loss}$.**



 ----

In [ ]:
def loss(X, y, beta, lambda_):
    ####### Complete this part ######## or die ####################
    ...
    return ...
    ###############################################################


Le code suivant permet d'afficher la fonction ou ses composantes qu'on cherche à optimiser.

In [ ]:
def plot(title, obj='loss', param_trace=None, lambda_=lambda_):
    plt.figure(figsize=(12.0, 8.0))
    ax = plt.gca()

    delta = 0.1
    x_range = np.arange(-20.0, 20.0+delta, delta)
    y_range = np.arange(-20.0, 20.0+delta, delta)
    XX, YY = np.meshgrid(x_range, y_range)

    ZZ = np.zeros(XX.shape)
    for i in range(XX.shape[0]):
        for j in range(XX.shape[1]):
            if obj == 'loss':
                ZZ[i, j] = np.sqrt(loss(X, y, np.array([[XX[i, j]], [YY[i, j]]]), lambda_))
            elif obj =='penalty':
                ZZ[i, j] = penality(np.array([[XX[i, j]], [YY[i, j]]]), lambda_)
            elif obj == 'objective':
                ZZ[i, j] = np.sqrt(main_objective(X, y, np.array([[XX[i, j]], [YY[i, j]]])))
                

    CS = ax.contour(XX, YY, ZZ)
    ax.clabel(CS, inline=True, fontsize=10)
    # ax.scatter(real_beta[0, :], real_beta[1, :])
    ax.set_title(title)
    ax.axhline(0, lw=0.5, color='red') # x = 0
    ax.axvline(0, lw=0.5, color='red') # y = 0
    lines = None
    if param_trace is not None:
        lines = ax.plot(param_trace[:, 0], param_trace[:, 1], color='blue')
    plt.show()

On observe clairemant dans l'affichage ci-dessous la non différentiabilité de $\lVert x\rVert_1$ en certains endroits.

In [ ]:
plot(obj='penalty', title='$\ell_1$ penalty ($\\lambda||\\beta||_1$)')


À l'inverse, notre objectif principal est clairement "smooth".

In [ ]:
plot(obj='objective', title='$\\frac{1}{2}||X\\beta-y||_2^2$')


Malheureusement, la combinaison des deux rend notre fonction non différentiable en certains endroits.

In [ ]:
plot(obj='loss', title='$\\frac{1}{2}||X\\beta-y||_2^2+\lambda||\\beta||_1$')


**<span style='color:blue'> Exercice</span>** 
**L'algorithme d'optimisation proximal s'appuie sur deux opérations. Le gradient et l'operateur proximal. Complétez les deux fonctions associées ci-dessous.**

**Ensuite, proposez la méthode d'optimisation de notre classe $\texttt{ProximalOptimization}$.**



 ----

In [ ]:
class ProximalOptimization(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def gradient(self, beta):
        ####### Complete this part ######## or die ####################
        return ...
        ###############################################################
    
    def gradient_descent_step(self, learning_rate, beta):
        ####### Complete this part ######## or die ####################
        ...
        return ...
        ###############################################################
    
    def proxy_step(self, learning_rate, lambda_, u):
        ####### Complete this part ######## or die ####################
        ...
        return ...
        ###############################################################
        
    def optimize(self, learning_rate, beta, lambda_, nb_iterations):
        ####### Complete this part ######## or die ####################
        ...
        return ...
        ###############################################################


**<span style='color:blue'> Exercice</span>** 
**Jouez avec les différents paramètres afin d'en observer les effets.**



 ----

In [ ]:
lambda_= 12
beta = np.array([[-1], [19]])
optim = ProximalOptimization(X, y)
param_trace = optim.optimize(0.01, beta, lambda_, 1000)
plot(
    obj='loss', title='Optimizing $\\frac{1}{2}||X\\beta-y||_2^2+\lambda||\\beta||_1$', 
    param_trace=param_trace
)


**<span style='color:blue'> Exercice</span>** 
**Qu'observez-vous lorsque notre chemin d'optimisation passe trop proche d'un axe ?**



 ----